In [1]:
!pip install gdown

In [ ]:
import gdown
file_id = '1-Be04_a1hzkmuV6u2X3ZH9n6uXt72Qcj'
destination = '15000_last_checkpoint.pth'
gdown.download(f'https://drive.google.com/uc?id={file_id}', destination, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1-Be04_a1hzkmuV6u2X3ZH9n6uXt72Qcj
From (redirected): https://drive.google.com/uc?id=1-Be04_a1hzkmuV6u2X3ZH9n6uXt72Qcj&confirm=t&uuid=e35049a4-02c5-4ac5-a972-5b7e4d2be8ea
To: /content/15000_last_checkpoint.pth
100%|██████████| 507M/507M [00:03<00:00, 131MB/s]


'15000_last_checkpoint.pth'

In [2]:
import gdown
file_id2 = '1-3kloR13lQQJSMTP-B5nuIygnZG0uITW'
destination = '80000_last_checkpoint.pth'
gdown.download(f'https://drive.google.com/uc?id={file_id2}', destination, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1-3kloR13lQQJSMTP-B5nuIygnZG0uITW
From (redirected): https://drive.google.com/uc?id=1-3kloR13lQQJSMTP-B5nuIygnZG0uITW&confirm=t&uuid=996ed1d6-d99d-4048-9bc4-6c90a52f0839
To: /content/80000_last_checkpoint.pth
100%|██████████| 507M/507M [00:06<00:00, 79.3MB/s]


'80000_last_checkpoint.pth'

In [ ]:
!python main.py

config.json: 100% 665/665 [00:00<00:00, 3.18MB/s]
model.safetensors: 100% 548M/548M [00:05<00:00, 96.5MB/s]
generation_config.json: 100% 124/124 [00:00<00:00, 733kB/s]
tokenizer_config.json: 100% 26.0/26.0 [00:00<00:00, 165kB/s]
vocab.json: 100% 1.04M/1.04M [00:00<00:00, 2.01MB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 3.43MB/s]
tokenizer.json: 100% 1.36M/1.36M [00:00<00:00, 4.17MB/s]
100%|███████████████████████████████████████| 338M/338M [00:08<00:00, 43.2MiB/s]
Traceback (most recent call last):
  File "/content/main.py", line 7, in <module>
    infer.query("/content/img.jpg","what is child doing ?")
  File "/content/inference.py", line 20, in query
    assert os.path.isfile(pathToImg) and type(textQuery) is str , "Expected Image path and query in form of text"
AssertionError: Expected Image path and query in form of text


In [3]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://uynhws1pt4r-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [4]:
!pip install openai-clip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 7.4 MB/s eta 0:00:00
  Created wheel for openai-clip: filename=openai_clip-1.0.1-py3-none-any.whl size=1368605 sha256=8d9b7f2c953881858e35fe386389ed68219007aff73801d47ead637fca0685f6
  Stored in directory: /root/.cache/pip/wheels/08/77/8e/8d2f862df6bf7fb4e2007062d2cbaeae49862ec7b56d041229
Successfully built openai-clip


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from flask import Flask, render_template, request, jsonify
import os
import importlib.util
import sys
import cv2

# Specify the module name and file path
module_name = "inference"
file_path = "/content/inference.py"

# Load the module
spec = importlib.util.spec_from_file_location(module_name, file_path)
module = importlib.util.module_from_spec(spec)
sys.modules[module_name] = module
spec.loader.exec_module(module)

from inference import Inferece
app = Flask(__name__, template_folder='/content/drive/MyDrive/templates')
inference = Inferece('/content/80000_last_checkpoint.pth')

def extract_frames_and_process(video_path, question):
    frames_folder = "/content/frames"
    extract_frames_at_intervals(video_path, frames_folder, 0.5)
    answers = {}
    for frame_file in sorted(os.listdir(frames_folder)):
        frame_path = os.path.join(frames_folder, frame_file)
        answer = inference.query(frame_path, question)
        time_stamp = frame_file.split('_')[1].split('.')[0]
        answers[time_stamp] = answer
    return answers

def extract_frames_at_intervals(video_path, output_folder, seconds_interval):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count / fps
    current_frame = 0
    frame_interval = int(fps * seconds_interval)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if current_frame % frame_interval == 0:
            output_file = os.path.join(output_folder, f"frame_{current_frame:04d}.jpg")
            cv2.imwrite(output_file, frame)
        current_frame += 1
    cap.release()

@app.route("/", methods=["GET", "POST"])
def home():
    if request.method == "POST":
        if 'file' not in request.files or 'question' not in request.form:
            return render_template('index.html', error="No file or question provided"), 400
        file = request.files['file']
        question = request.form['question']
        file_path = "/content/tmp_file"
        file.save(file_path)
        if file.filename.endswith('.mp4'):
            answers = extract_frames_and_process(file_path, question)
            os.remove(file_path)
            return render_template('index.html', answers=answers)
        else:
            answer = inference.query(file_path, question)
            os.remove(file_path)
            return render_template('index.html', image_answer=answer)
    return render_template('index.html')

if __name__ == "__main__":
    app.run()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

100%|███████████████████████████████████████| 338M/338M [00:13<00:00, 26.9MiB/s]


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
